In [1]:
!pip install -q git+https://github.com/pytube/pytube
!pip install -q telebot moviepy savify

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.2/232.2 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.8/251.8 kB 10.8 MB/s eta 0:00:00


In [3]:

import telebot
from pytube import YouTube
import moviepy.editor as mp

TOKEN = '6821987683:AAE25CP83E8G4pPAvUFVlfwKKuz6aJ_Kw1U' #@param {type: 'string'}

bot = telebot.TeleBot(TOKEN)

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Welcome to Video to Audio Converter Bot! Send me a YouTube link.")

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    url = message.text
    yt = YouTube(url)
    video_stream = yt.streams.get_highest_resolution()

    markup = telebot.types.ReplyKeyboardMarkup(one_time_keyboard=True)
    markup.add(telebot.types.KeyboardButton('Download Video'), telebot.types.KeyboardButton('Convert to Audio'))

    bot.reply_to(message, "Choose an option:", reply_markup=markup)

    bot.register_next_step_handler(message, handle_option, url, video_stream)

def handle_option(message, url, video_stream):
    option = message.text.lower()

    if option == 'download video':
        video_path = video_stream.download()
        bot.reply_to(message, "Video downloaded successfully! Sending the video file...")
        sent_message = bot.send_video(message.chat.id, open(video_path, 'rb'))
    elif option == 'convert to audio':
        video_path = video_stream.download()
        video = mp.VideoFileClip(video_path)
        audio = video.audio
        audio_path = video_path.replace('.mp4', '.mp3')
        audio.write_audiofile(audio_path)
        video.close()
        audio.close()
        bot.reply_to(message, "Audio conversion completed successfully! Sending the audio file...")
        sent_message = bot.send_audio(message.chat.id, open(audio_path, 'rb'))
    else:
        bot.reply_to(message, "Invalid option. Please choose 'Download Video' or 'Convert to Audio'.")

bot.polling()

RegexMatchError: regex_search: could not find match for (?:v=|\/)([0-9A-Za-z_-]{11}).*